In [ ]:
from skimage.io import imread
from skimage import morphology, measure
import os, glob, tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', size=15)
rc('font', family='Arial')
# rc('font', serif='Times')
rc('pdf', fonttype=42)
# rc('text', usetex=True)


First, define the experiment folder (it depends on whether you are using local machine or server workstation).
So far, the analysis can be run on a single well of a single ibidi slide at a time. In this case we chose 'coated-wt-triton'

In [ ]:
pc = os.environ['COMPUTERNAME']
if pc=='PCBA-TRIVEDI03': # my Razer
    folder_raw = os.path.join('E:',os.sep,'immuno_NMG')
elif pc=='PCBA-TRIVEDI02': # workstation
    folder_raw = os.path.join('Y:',os.sep,'Nicola_Gritti','raw_data','immuno_NMG')

exp_folder = os.path.join('2020-12-01_NGM_immuno_tests')

resultFolder = os.path.join(folder_raw, exp_folder, 'results')

down=1
imgName = 'rep6_WT_noTrim-stitched_%d%d.tif'%(down,down)

Next, define which channels are available in the images and the color you want to have them displayed with.
In this case, the membrane marker is not present because we remove it at stitching stage...

In [ ]:
channels = ['Bright field','DAPI','Sepdb1']
colors = ['black','blue','orange']

Read in the csv file containing the properties of the cells from the StarDist segmentation and skimage-measureprops function (that is, from the previous script).
Also, load the full image and the mask generated by the segmentation

In [ ]:
# load cell props
print('Loading cell props...')
props = pd.read_csv(os.path.join(resultFolder,'props_'+os.path.splitext(imgName)[0]+'.csv'))
nch = len([i for i in props.keys() if 'mean_intens' in i])
print('Done. Found %d cells with %d features'%(len(props.label), len(props.keys())))

# load image
print('Loading image...')
X = imread(os.path.join(resultFolder,imgName))
print('Done.',X.shape)

# load image
print('Loading mask...')
maskName = 'mask_'+imgName
Y = imread(os.path.join(resultFolder,maskName))
print('Done.',Y.shape)


In [ ]:
props.head()

In [ ]:
print(props.keys())
print('Cells:', len(props.label))

In [ ]:
# plot of distributions
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(6,9))
fig.subplots_adjust(top=0.99, hspace=0.5, bottom=0.1, right=0.99)
for i in range(nch):
    data = props['mean_intensity-%d'%i]
    data = np.clip(data,0,np.percentile(data, 99.7))
    ax[i].hist(data, bins=100, color=colors[i])
    ax[i].set_xlabel('Channel %d (%s)'%(i+1,channels[i]))
    ax[i].set_ylabel('Num cells')
    ax[i].set_yscale('log')
plt.show()

fig.savefig(os.path.join(resultFolder, os.path.splitext(imgName)[0].split('-')[0] + '_histogram.png'),dpi=300)

In [ ]:
# determine cutoff to define cells that were successfully transfected
# use the gfp channel to determine threshold.
gfp = props['mean_intensity-%d'%2].values
dapi = props['mean_intensity-%d'%1].values

# Assume bimodal distribution (negative cells + positive cells) and use KMeans clustering with 2 clusters
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)

# cluster the data points
kmeans.fit(gfp.reshape(-1,1))
y_kmeans = kmeans.predict(gfp.reshape(-1,1))

# custom threshold
y_kmeans = (gfp>500)&(gfp<10000)

# visualize
from matplotlib.colors import LinearSegmentedColormap
colors = [(0.0, 0.0, 0.0), (0,0,.5), (.2, .2, 1.), (.1,1.,1.)] # Experiment with this
cm = LinearSegmentedColormap.from_list('test', colors, N=255)

fig, ax = plt.subplots()
ax.scatter(dapi, gfp, c=y_kmeans, s=10, cmap=cm)
plt.yscale('log')
ax.set_xlabel('DAPI')
ax.set_ylabel('GFP')

# assign cluster to cells
props['transfected'] = y_kmeans.astype(bool)
ax.text(np.max(props['mean_intensity-1'])*2/3,10000,'Tr:'+str(len(props[props.transfected])))
ax.text(np.max(props['mean_intensity-1'])*2/3,6000,'noTr:'+str(len(props[props.transfected==False])))

print('Transfected cells:', len(props[props.transfected]))
print('Not transfected cells:', len(props[props.transfected==False]))

fig.savefig(os.path.join(resultFolder, os.path.splitext(imgName)[0].split('-')[0] + '_transfected.png'),dpi=300)

In [ ]:
transfected = props[props.transfected]
transfected.head()

In [ ]:
n_sample = 20
not_transfected_sample = props[[not i for i in props.transfected.values]].sample(n=n_sample) 

fig,ax=plt.subplots(n_sample,len(channels)+4,figsize=(15,2*n_sample))
plt.subplots_adjust(hspace=0.2,right=0.99)

i=0
for idx, cell in not_transfected_sample.iterrows():
    cell_img = X[int(cell['bbox-0']):int(cell['bbox-2']),int(cell['bbox-1']):int(cell['bbox-3']),:]
    cell_img = np.moveaxis(cell_img, -1, 0)
    
    cell_mask = Y[int(cell['bbox-0']):int(cell['bbox-2']),int(cell['bbox-1']):int(cell['bbox-3'])]
    cell_mask = np.pad(cell_mask,1)
    cell_mask = morphology.binary_erosion(cell_mask==cell.label,morphology.disk(4))
    cell_mask = cell_mask[1:-1,1:-1]

    for j in range(len(channels)):
        ax[i,j].imshow(cell_img[j], cmap='gray')#,clim=(0,5000))
    ax[i,len(channels)].imshow(cell_mask, cmap='gray')
    ax[i,0].set_ylabel(i)

    dapi = cell_img[1].flatten()
    dapi = dapi[cell_mask.flatten()==True]
    dapi = (dapi-np.mean(dapi))/np.mean(dapi)
    
    trim28 = cell_img[2].flatten()
    trim28 = trim28[cell_mask.flatten()==True]
    trim28 = (trim28-np.mean(trim28))/np.mean(trim28)

    sepdb1 = cell_img[3].flatten()
    sepdb1 = sepdb1[cell_mask.flatten()==True]
    sepdb1 = (sepdb1-np.mean(sepdb1))/np.mean(sepdb1)
    
    ax[i,len(channels)+1].scatter(dapi,sepdb1)
    ax[i,len(channels)+1].set_xlabel('DAPI')
    ax[i,len(channels)+1].set_ylabel('Sepdb1')
    ax[i,len(channels)+1].text(0,0,'%.04f'%np.corrcoef(dapi,sepdb1)[0,1])

    ax[i,len(channels)+2].scatter(dapi,trim28)
    ax[i,len(channels)+2].set_xlabel('DAPI')
    ax[i,len(channels)+2].set_ylabel('Trim28')
    ax[i,len(channels)+2].text(0,0,'%.04f'%np.corrcoef(dapi,trim28)[0,1])

    ax[i,len(channels)+3].scatter(trim28,sepdb1)
    ax[i,len(channels)+3].set_xlabel('Trim28')
    ax[i,len(channels)+3].set_ylabel('Sepdb1')
    ax[i,len(channels)+3].text(0,0,'%.04f'%np.corrcoef(trim28,sepdb1)[0,1])

    i+=1

for j in range(len(channels)):
    ax[0,j].set_title(channels[j])
#     plt.axis('off')
ax[0,len(channels)].set_title('Mask')

# remove the x and y ticks
for a in ax.flatten():
    a.set_xticks([])
    a.set_yticks([])

# fig.savefig(os.path.join(resultFolder, os.path.splitext(imgName)[0].split('-')[0] + '_exampleImages.png'),dpi=300)

In [ ]:
n_sample = 20
transfected_sample = transfected.sample(n=n_sample) 

fig,ax=plt.subplots(n_sample,len(channels)+4,figsize=(15,2*n_sample))

i=0
for idx, cell in transfected_sample.iterrows():
    cell_img = X[int(cell['bbox-0']):int(cell['bbox-2']),int(cell['bbox-1']):int(cell['bbox-3']),:]
    cell_img = np.moveaxis(cell_img, -1, 0)
    
    cell_mask = Y[int(cell['bbox-0']):int(cell['bbox-2']),int(cell['bbox-1']):int(cell['bbox-3'])]
    cell_mask = np.pad(cell_mask,1)
    cell_mask = morphology.binary_erosion(cell_mask==cell.label,morphology.disk(4))
    cell_mask = cell_mask[1:-1,1:-1]

    for j in range(len(channels)):
        ax[i,j].imshow(cell_img[j], cmap='gray')#,clim=(0,5000))
    ax[i,len(channels)].imshow(cell_mask, cmap='gray')
    ax[i,0].set_ylabel(i)

    dapi = cell_img[1].flatten()
    dapi = dapi[cell_mask.flatten()==True]
    dapi = (dapi-np.mean(dapi))/np.mean(dapi)
    
    trim28 = cell_img[2].flatten()
    trim28 = trim28[cell_mask.flatten()==True]
    trim28 = (trim28-np.mean(trim28))/np.mean(trim28)

    sepdb1 = cell_img[3].flatten()
    sepdb1 = sepdb1[cell_mask.flatten()==True]
    sepdb1 = (sepdb1-np.mean(sepdb1))/np.mean(sepdb1)
    
    ax[i,len(channels)+1].scatter(dapi,sepdb1)
    ax[i,len(channels)+1].set_xlabel('DAPI')
    ax[i,len(channels)+1].set_ylabel('Sepdb1')
    ax[i,len(channels)+1].text(0,0,'%.04f'%np.corrcoef(dapi,sepdb1)[0,1])

    ax[i,len(channels)+2].scatter(dapi,trim28)
    ax[i,len(channels)+2].set_xlabel('DAPI')
    ax[i,len(channels)+2].set_ylabel('Trim28')
    ax[i,len(channels)+2].text(0,0,'%.04f'%np.corrcoef(dapi,trim28)[0,1])

    ax[i,len(channels)+3].scatter(trim28,sepdb1)
    ax[i,len(channels)+3].set_xlabel('Trim28')
    ax[i,len(channels)+3].set_ylabel('Sepdb1')
    ax[i,len(channels)+3].text(0,0,'%.04f'%np.corrcoef(trim28,sepdb1)[0,1])

    i+=1

for j in range(len(channels)):
    ax[0,j].set_title(channels[j])
#     plt.axis('off')
ax[0,len(channels)].set_title('Mask')

# remove the x and y ticks
for a in ax.flatten():
    a.set_xticks([])
    a.set_yticks([])

# fig.savefig(os.path.join(resultFolder, os.path.splitext(imgName)[0].split('-')[0] + '_exampleImages.png'),dpi=300)

In [ ]:
corr_12 = [np.nan for i in range(len(props))] # dapi-trim28
corr_13 = [np.nan for i in range(len(props))] # dapi-sepdb1
corr_23 = [np.nan for i in range(len(props))] # trim28-sepdb1

i=0
for idx, cell in tqdm.tqdm(props.iterrows(), total=len(props.label)):
    cell_img = X[int(cell['bbox-0']):int(cell['bbox-2']),int(cell['bbox-1']):int(cell['bbox-3']),:]
    cell_img = np.moveaxis(cell_img, -1, 0)
    
    cell_mask = Y[int(cell['bbox-0']):int(cell['bbox-2']),int(cell['bbox-1']):int(cell['bbox-3'])]
    cell_mask = np.pad(cell_mask,1)
    cell_mask = morphology.binary_erosion(cell_mask==cell.label,morphology.disk(4))
    cell_mask = cell_mask[1:-1,1:-1]

    dapi = cell_img[1].flatten()
    dapi = dapi[cell_mask.flatten()==True]
    dapi = (dapi-np.mean(dapi))/np.mean(dapi)
    
    trim28 = cell_img[2].flatten()
    trim28 = trim28[cell_mask.flatten()==True]
    trim28 = (trim28-np.mean(trim28))/np.mean(trim28)
    
    sepdb1 = cell_img[3].flatten()
    sepdb1 = sepdb1[cell_mask.flatten()==True]
    sepdb1 = (sepdb1-np.mean(sepdb1))/np.mean(sepdb1)

    corr_12[i] = np.corrcoef(dapi,trim28)[0,1]
    corr_13[i] = np.corrcoef(dapi,sepdb1)[0,1]
    corr_23[i] = np.corrcoef(trim28,sepdb1)[0,1]
        
    i+=1

# append data to dataframe
props['corr_12'] = corr_12
props['corr_13'] = corr_13
props['corr_23'] = corr_23
    

In [ ]:
# for the sample, show correlation plots
import seaborn as sns
df = props[props.transfected]
df = df[['corr_12','corr_13','corr_23']]

fig,ax=plt.subplots(figsize=(6,6))
fig.subplots_adjust(left=0.2,hspace=0.5,wspace=0.3)

sns.violinplot(x="variable", y="value", data=pd.melt(df), ax=ax, palette=['royalblue','limegreen','indianred'], inner='quartile')#, showfliers=False)

# sns.boxplot(x="variable", y="value", data=pd.melt(df), ax=ax, showfliers=False, palette=['royalblue','limegreen','indianred'])
# sns.swarmplot(x="variable", y="value", data=pd.melt(df), ax=ax, s=1, color='black')

ax.set_ylim(-1,1)
ax.plot([-0.5,2.5],[0,0],'--k')
ax.set_xlim(-0.5,2.5)

ax.text(0,-0.9,'%.3f'%np.mean(df['corr_12']))
ax.text(1,-0.9,'%.3f'%np.mean(df['corr_13']))
ax.text(2,-0.9,'%.3f'%np.mean(df['corr_23']))
plt.show()

print('DAPI-Trim28:', np.mean(df['corr_12']), np.std(df['corr_12']))
print('DAPI-Spedb1:', np.mean(df['corr_13']), np.std(df['corr_13']))
print('Trim28-Sepdb1:', np.mean(df['corr_23']), np.std(df['corr_23']))

fig.savefig(os.path.join(resultFolder, os.path.splitext(imgName)[0].split('-')[0] + '_corrTransfected.png'),dpi=300)

In [ ]:
# for the sample, show correlation plots
import seaborn as sns
df = props[props.transfected==False]
df = df[['corr_12','corr_13','corr_23']]

fig,ax=plt.subplots(figsize=(6,6))
fig.subplots_adjust(left=0.2,hspace=0.5,wspace=0.3)

sns.violinplot(x="variable", y="value", data=pd.melt(df), ax=ax, palette=['royalblue','limegreen','indianred'], inner='quartile')#, showfliers=False)
# sns.boxplot(x="variable", y="value", data=pd.melt(df), ax=ax, showfliers=False, palette=['royalblue','limegreen','indianred'])
# sns.swarmplot(x="variable", y="value", data=pd.melt(df), ax=ax, s=2, color='black')

ax.set_ylim(-1,1)
ax.plot([-0.5,2.5],[0,0],'--k')
ax.set_xlim(-0.5,2.5)

ax.text(0,-0.9,'%.3f'%np.mean(df['corr_12']))
ax.text(1,-0.9,'%.3f'%np.mean(df['corr_13']))
ax.text(2,-0.9,'%.3f'%np.mean(df['corr_23']))
plt.show()

print('DAPI-Trim28:', np.mean(df['corr_12']), np.std(df['corr_12']))
print('DAPI-Spedb1:', np.mean(df['corr_13']), np.std(df['corr_13']))
print('Trim28-Sepdb1:', np.mean(df['corr_23']), np.std(df['corr_23']))

fig.savefig(os.path.join(resultFolder, os.path.splitext(imgName)[0].split('-')[0] + '_corrNotTransfected.png'),dpi=300)

In [ ]:

fig,ax=plt.subplots(figsize=(6,6))
fig.subplots_adjust(left=0.2,hspace=0.5,wspace=0.3)

not_trasnfected = props[props.transfected==False]
ax.scatter(not_trasnfected['corr_12'],not_trasnfected['mean_intensity-2'],color='black')
transfected = props[props.transfected==True]
ax.scatter(transfected['corr_12'],transfected['mean_intensity-2'],color='red')
ax.set_xlabel('corr trim28-sepdb1')
ax.set_ylabel('GFP')
ax.set_yscale('log')
ax.set_xlim(-1,1)
ax.set_ylim(1e2, 1e5)

fig.savefig(os.path.join(resultFolder, os.path.splitext(imgName)[0].split('-')[0] + '_corrVsGFP.png'),dpi=300)